<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_04_4_persistence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 4: LangChain: Chat and Memory**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 4 Material

* Part 4.1: LangChain Conversations [[Video]]() [[Notebook]](t81_559_class_04_1_langchain_chat.ipynb)
* Part 4.2: Conversation Buffer Window Memory [[Video]]() [[Notebook]](t81_559_class_04_2_memory_buffer.ipynb)
* Part 4.3: Chat with Summary and Fixed Window [[Video]]() [[Notebook]](t81_559_class_04_3_summary.ipynb)
* **Part 4.4: Chat with Persistence, Rollback and Regeneration** [[Video]]() [[Notebook]](t81_559_class_04_4_persistence.ipynb)
* Part 4.5: Automated Coder Application [[Video]]() [[Notebook]](t81_559_class_04_5_coder.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai
    !wget -q https://raw.githubusercontent.com/jeffheaton/app_generative_ai/main/util_chat.py -O util_chat.py

Note: using Google CoLab


# 4.4: Persistence, Rollback and Regeneration

## Introducing the ChatConversation Class  

In this module we introduce the `ChatConversation` class, which builds on the foundation of the earlier `SimpleConversation` wrapper. The original version provided a straightforward way to send prompts, display responses, and track conversation history. While useful for lightweight experimentation, it was limited in how history was managed and offered few tools for editing or persisting sessions. The new `ChatConversation` class expands this design into a more flexible and production-ready framework.  

The key innovation of `ChatConversation` is its use of pluggable message strategies. Instead of always storing the full conversation transcript, you can now choose from different strategies such as a passthrough mode that keeps everything, a fixed window that trims to the most recent messages, or a summarization strategy that compresses older content into a running summary. Each strategy is modular, registered in a common base class, and can be swapped in or out as needed. This provides a clean way to experiment with different approaches to memory without altering the main conversation logic.  

Another major enhancement is the addition of editing controls. With `ChatConversation`, you are no longer locked into a strict linear progression of user and AI turns. You can undo the most recent interaction or regenerate an answer to a prior input. These tools make it easier to recover from mistakes, test variations of outputs, and maintain a smooth conversation flow. Logging is integrated into these operations to provide transparency and traceability.  

Finally, `ChatConversation` introduces the ability to persist and restore conversations. You can save a session’s state to disk, capturing both the history and the active strategy configuration. Later, you can reload the conversation and resume where you left off. This persistence feature is especially valuable for experiments, reproducibility, and applications that need continuity across sessions. Together, these improvements make `ChatConversation` a powerful and adaptable tool for managing conversational AI workflows.  

For convenience, we place the `ChatConversation` class and its supporting strategies into a separate Python file named `util_chat.py`. This allows us to keep the module organized and makes it easy to reuse the functionality in notebooks or other projects. Once saved, you can simply bring it into your workflow with:  


In [ ]:
import util_chat

We can now carry on a simple conversation with the LLM, using LangChain to track the conversation memory.

We will start by looking at a few examples that demonstrate how the new class works with different memory strategies. In particular, we will compare the buffer approach, which keeps the most recent turns verbatim, with the summary approach, which compresses older context into a running synopsis. These examples will highlight how each strategy affects what the model remembers and how the conversation evolves over time.  

### Example: Fixed-window “buffer” memory

The following example demonstrates how to use the fixed-window strategy to preserve memory across sessions. We begin by starting a conversation, providing some context, and saving the state to disk. Next, we create a new session with a different configuration and load the saved memory. Finally, we verify that the model recalls the previously stored fact, showing how persistence works with the buffer approach.  


In [ ]:
# --- Fixed-window (buffer) example ---
# Assumes the ChatConversation and strategies code from before is already defined in the notebook.

# 1) Start a session, teach a fact, and save memory
conv_fixed_1 = util_chat.ChatConversation(
    strategy_name="fixed",
    strategy_kwargs={"window": 14, "keep_system_first": True},
)
conv_fixed_1.chat("Hi, my name is Jeff.")
conv_fixed_1.chat("Please remember my favorite color is blue.")
conv_fixed_1.save("mem_fixed.json")

# 2) New session (could be a new process), load memory, and verify it works
conv_fixed_2 = util_chat.ChatConversation(
    strategy_name="fixed",
    strategy_kwargs={"window": 8, "keep_system_first": True},  # different config is fine
)

# Load the saved memory into this session
conv_fixed_2.load("mem_fixed.json")

# Ask a question to prove memory came back
resp = conv_fixed_2.chat("What is my favorite color?")

**Human:** 

Hi, my name is Jeff.

**AI:** 

Hi Jeff — nice to meet you. How can I help you today?

**Human:** 

...

Got it, Jeff — I’ll remember that your favorite color is blue for the rest of this conversation. Would you like me to use that now (for example, when suggesting colors or examples), or save any other preferences?

**Human:** 

What is my favorite color?

**AI:** 

Your favorite color is blue. Would you like me to remember that across future conversations too?

After saving the conversation state, we can inspect the JSON file directly to see what was stored. The command below shows the first few lines of the saved file, giving us a look at how the strategy, messages, and session details are represented.  


In [ ]:
!head mem_fixed.json

{
  "version": 1,
  "session_id": "5c02d3e2-e8d7-4ed4-8f8e-f864f5763522",
  "strategy_state": {
    "strategy_name": "fixed",
    "messages": [
      {
        "type": "human",
        "data": {
          "content": "Hi, my name is Jeff.",


### Example: Summarizing memory

This next example shows how the summarizing strategy works in practice. Instead of keeping every message in memory, older turns are compressed into a running summary while the most recent exchanges remain verbatim. We begin by starting a session, teaching the model a fact, and adding enough small talk to trigger summarization. After saving the session, we load it into a new conversation and confirm that the important detail still survives through the summary mechanism.  


In [ ]:
# --- Summarizing strategy example ---
# Assumes the ChatConversation and strategies code from before is already defined in the notebook.

# 1) Start a session with summary strategy, teach a fact
conv_sum_1 = util_chat.ChatConversation(
    strategy_name="summary",
    strategy_kwargs={
        "keep_last": 8,
        "trigger_len": 12,
        "max_summary_chars": 800,
        "system_prefix": "Running summary",
    },
)
conv_sum_1.chat("Hello assistant.")
conv_sum_1.chat("Please remember my dog's name is Hickory.")

# Add enough chatter to trigger summarization in normal use
for i in range(16):
    conv_sum_1.chat(f"Small talk line #{i}: noting something irrelevant. Just say OK if you got it.")

# Save after some turns (summary strategy persists both the raw tail and the compact summary)
conv_sum_1.save("mem_summary.json")

# 2) New session (could be a new process), load memory, and verify it works
conv_sum_2 = util_chat.ChatConversation(
    strategy_name="summary",
    strategy_kwargs={"keep_last": 6, "trigger_len": 10},  # different config is fine
)

# Load the saved memory into this session
conv_sum_2.load("mem_summary.json")

# Ask a question to prove the fact survived via the running summary
resp = conv_sum_2.invoke("What is my dog's name?")
print("AI:", resp.content)  # Expect: "Hickory" (or equivalent)

**Human:** 

Hello assistant.

**AI:** 

Hello! How can I help you today?

**Human:** 

...

**Human:** 

Small talk line #15: noting something irrelevant. Just say OK if you got it.

**AI:** 

OK

AI: Your dog's name is Hickory.


Just like before, we can take a look at the saved JSON file to see how the summarizing strategy records its state. The command below prints the first few lines, which will include both the compact summary and the recent tail of the conversation.  


In [ ]:
!head mem_summary.json

{
  "version": 1,
  "session_id": "8ae9df68-cbda-4a94-9b62-6c8853d1c270",
  "strategy_state": {
    "strategy_name": "summary",
    "messages": [
      {
        "type": "human",
        "data": {
          "content": "Hello assistant.",


### Undo and Regenerate

The final example demonstrates the default passthrough strategy, which keeps the entire conversation history without trimming or summarizing. In this case, we also explore the editing controls that `ChatConversation` provides. We start by introducing some context, then use `undo()` to remove the most recent interaction. Afterward, we continue the conversation and finally call `regenerate()` to replace the last AI response with a fresh one. This shows how you can manage and refine conversations interactively while retaining full history.  


In [ ]:
# Create the conversation with default passthrough strategy
c = util_chat.ChatConversation(system_prompt="You are a helpful assistant that answers in single sentences.")
print("---")
c.chat("Hello, my name is Jeff.")
print("---")
c.chat("I like coffee.")
print("---")
c.undo()
print("---")
c.chat("What is my name.")
print("---")
c.chat("Do I like coffee?")
c.regenerate()


---


**Human:** 

Hello, my name is Jeff.

**AI:** 

Hi Jeff, nice to meet you—how can I help today?

...

I don't know whether you like coffee, Jeff—do you?

**Human (regenerated):** 

Do I like coffee?

**AI (regenerated):** 

I don't know whether you like coffee—do you?